In [9]:
import os
import sys; sys.path.append("../../../")
from img_pipe.img_pipe import img_pipe
from importlib import reload
reload(img_pipe)
import numpy as np
import scipy.io
%matplotlib inline

# Img_pipe Recon Demo

This tutorial will walk through how to run a recon using the img_pipe package. This includes MRI alignment, CT registration, electrode localization, anatomical labeling, and nonlinear warping. 

### Set-up

In your SUBJECTS_DIR (where you want to place your subjects imaging data), you should make a new directory (with your subject name, i.e. 'EC86'). Then, create subdirectories __*acpc/*__ and __*CT/*__. Place your niftii format T1 MRI file (name it *T1_orig.nii*) in the __*acpc/*__ folder and the CT niftii file (name it *CT.nii*) in __*CT/*__. Your directory structure should now look like this: 

SUBJECTS_DIR/
    * Subj_name/
        * acpc/
            *T1_orig.nii
        * CT/
            * CT.nii

### 1. Alignment of T1 scan to AC-PC axis

*	Alignment of the T1 scan to the anterior commissure-posterior commissure axis is performed in Freeview (Figure 3). Open Freeview and load your unaligned T1 scan in the Volumes tab. To aid in axis alignment, change the cursor style “long” in Preferences  Cursor  style “Long”. You can also change the color if you wish.
*	To adjust the rotation and translation of the image, select Tools  Transform Volume. Adjust the roll (with Y (P-A)) and yaw (with Z (I-S)) as necessary to make sure the head is aligned.  Check the axial view to make sure the eyes show equally in the same slice (see Fig 3A vs. 3B, second panel for unaligned and aligned examples).  Make sure the midsagittal line is vertical in the axial view (see Fig 3A vs. 3B, first and third panels) and in the coronal view.  Choose Sample method “Cubic”.
*	Select the anterior commissure and adjust the pitch of the head so that it is in line with the posterior commissure on the horizontal axis (Fig 3A-B, last panel).
*	Finally, move to the (0, 0, 0) RAS coordinate (not TkReg RAS, just RAS).  In the Transform Volume tool, translate the image until the cursor is at the anterior commissure.   
*	Once you are satisfied that the brain is in a good orientation, click “Save Reg…” and save the transformation matrix in the acpc directory as T1_reorient.lta. Then, click “Save as…” and save the reoriented T1 file as T1.nii in the acpc directory (e.g. /usr/local/freesurfer/subjects/EC1/acpc). 

#### Include image here 

### 2. Create the freeCoG instance for the patient and run preparatory steps

In [10]:
patient = img_pipe.freeCoG(subj='umf004',
                           hem='rh',
                           subj_dir='/home/adam2392/hdd/data/neuroimaging/freesurfer_output/',
                           fs_dir='/opt/freesurfer/')
# you can also specify SUBJECTS_DIR and FREESURFER_HOME in your .bash_profile instead of specifying 
# in the *subj_dir* and *fs_dir* arguments.

#### 2a) **`prep_recon()`**
**`prep_recon()`** will set up the directory structure needed before running **`get_recon()`**

In [3]:
patient.prep_recon()

Making electrodes directory


#### 2b) **`get_recon()`** 
**`get_recon()`** will run the Freesurfer command `recon-all`. This will take on the order of hours, depending on your machine's CPU, GPU, and whether you use parallelization.

In [14]:
patient.get_recon()

#### 2c) check pial surfaces against the MRI with **`check_pial()`**
Now, check that the pial surfaces actually correspond to the MRI.

In [13]:
#this will open a freesurfer window with the brain MRI and pial surfaces loaded
patient.check_pial()

#### 2d) create the meshes

Create the triangle-vertex mesh in .mat format with **`convert_fsmesh2mlab()`**. This will save them out to **__Meshes/EC108_lh_pial.mat__** and **__Meshes/EC108_rh_pial.mat__**. To create subcortical meshes, run **`get_subcort()`**, subcortical meshes will be in **__Meshes/subcortical/__**.

In [4]:
patient.convert_fsmesh2mlab()
patient.get_subcort()

Making Meshes Directory
Making lh mesh
Making rh mesh
::: Tesselating freesurfer subcortical segmentations from aseg using aseg2srf... :::
/home/adam2392/Documents/neuroimg_pipeline/img_pipe/img_pipe/SupplementalScripts/aseg2srf.sh
Creating directory /home/adam2392/hdd/data/neuroimaging/freesurfer_output/umf004/Meshes/subcortical
::: Converting all ascii segmentations to matlab tri-vert :::


#### 2e) CT Registration

Register the **__CT/CT.nii__** file to **__mri/orig.nii__** file; the registered CT will be saved as **__CT/rCT.nii__**.

Note, that the coregistred CT can just be copy and pasted over from our flirt run. This coregistratino using freesurfer is not that great.

In [27]:
# patient.reg_img('CT.nii','T1.nii')

Computing registration from /home/adam2392/hdd/data/neuroimaging/freesurfer_output/umf001/CT/CT.nii to /home/adam2392/hdd/data/neuroimaging/freesurfer_output/umf001/mri/T1.nii
Initial guess...
translation : [0. 0. 0.]
rotation    : [0. 0. 0.]

Optimizing using fmin_powell
translation : [1. 1. 1.]
rotation    : [0.01 0.01 0.01]

nmi = 2.0

Optimization terminated successfully.
         Current function value: -2.000000
         Iterations: 1
         Function evaluations: 25
Saving registered CT image as /home/adam2392/hdd/data/neuroimaging/freesurfer_output/umf001/CT/rCT.nii


/home/adam2392/.local/lib/python3.6/site-packages/nipy/io/files.py:145: FutureWarning: Default `strict` currently False; this will change to True in a future version of nipy
  ni_img = nipy2nifti(img, data_dtype = io_dtype)


### 3. Electrode localization, anatomical labeling, and nonlinear warping

#### 3a) Marking the electrodes on the CT

Example of identification of electrode coordinates using electrode picker, opened with **`mark_electrodes()`**. 

<img src='files/ElectrodeMarker.png' style='width:700px;'>

**(A)** Demonstrates the process of picking the coordinate for the most posterior inferior grid corner. On the left, the GUI is shown with the electrode selected. The pial surface, rCT, and skull stripped MRI are displayed. The upper left shows the electrode selected in the sagittal view. The upper right shows the coronal view. The bottom right shows an axial view. The lower left displays the intensity projection map of the CT, which is useful for visualizing the entire grid. To save the coordinate, press ‘n’ to name a new device. With the center of the electrode artifact localized by the crosshairs in the axial, sagittal, and coronal views, press ‘e’ to add a point. The coordinates are automatically saved to the ‘elecs’ folder. This file can be loaded and the saved points can be plotted on the 3D surface mesh in MATLAB. This plot can be seen in the right panel. If the coordinates appear buried in the Mesh due to post operative brain shift, additional steps can be taken to project the electrode to the surface. 

**(B)** Example of identification of an electrode that is part of a subtemporal strip. The strip can be seen in the rCT.nii intensity projection map in the lower right panel. The coordinate is recorded from the center of the electrode artifact, seen in sagittal, coronal, and axial views. This coordinate can then be visualized on the 3D surface mesh in MATLAB, seen in the right panel.

### Note:
You need to name the hd_grid with <name>_corners.mat, which will then be able to be interpreted by patient.interp_grid


In [12]:
#This will open a GUI where you can click to mark the electrodes on the registered CT 
patient.mark_electrodes()

Launching electrode picker


#### 2f) Interpolate electrode grids

If you marked any grid corners, you should interpolate then project the electrodes.

In [21]:
help(patient.interp_grid)
patient.interp_grid(grid_basename='g',
                    ncols=8, nrows=6
                   )

Help on method interp_grid in module img_pipe.img_pipe.img_pipe:

interp_grid(nrows=16, ncols=16, grid_basename='hd_grid') method of img_pipe.img_pipe.img_pipe.freeCoG instance
    Interpolates corners for an electrode grid
    given the four corners (in order, 1, 16, 241, 256), or for
    32 channel grid, 1, 8, 25, 32.
    
    Parameters
    ----------
    nrows : int
        Number of rows in the grid
    ncols : int
        Number of columns in the grid
    grid_basename : str
        The base name of the grid (e.g. 'hd_grid' if you have a corners file
        called hd_grid_corners.mat)



#### 2g) Project electrodes that are buried under the cortical surface

This can either work on interpolated grids or individual electrodes. For interpolated grids, you have the option of using the mean normal vector as the projection direction (see figure). For individual electrodes, you can specify the projection direction if you set **`use_mean_normal=False`**. You can also select whether to use the dural surface or pial surface as the surface to project to.

<img src='files/ElectrodeProjection.png' style="width: 650px;">

**A.**  The grid’s corner electrodes are manually located. We interpolate the locations of the rest of the grid electrodes using these corner coordinates, giving us the electrode grid shown in red. The green arrows are the four normal vectors calculated from the corners, and the black arrow is the mean of those normal vectors and will act as our projection direction. 

**B.** Projection of the interpolated grid (red) to the convex hull of the pial surface (blue) using the mean normal vector (black arrow). The final projected electrode grid is shown in blue.


In [3]:
help(patient.project_electrodes)

Help on method project_electrodes in module img_pipe_chang.img_pipe.img_pipe:

project_electrodes(elecfile_prefix='hd_grid', use_mean_normal=True, surf_type='dural', num_iter=30, dilate=0.0, grid=True, convex_hull=True) method of img_pipe_chang.img_pipe.img_pipe.freeCoG instance
    Project electrodes to the brain's surface to correct for deformation.
    
    By default, projects the electrodes of a grid based on the mean normal vector of the four grid
    corner electrodes that were manually localized from the registered CT. Can also project strips
    and individual electrodes if grid=False (you will be prompted for a projection direction).
    
    Parameters
    ----------
    elecfile_prefix : str, optional
        prefix of the .mat file with the electrode coordinates matrix 
    use_mean_normal : bool, optional
        whether to use mean normal vector (mean of the 4 normal vectors from the grids 
        corner electrodes) as the projection direction
    surf_type : {'dural','

In [13]:
patient.project_electrodes(elecfile_prefix='llat', grid=False)

Projection Params: 
	 Grid Name: llat.mat 
	 Use Mean Normal: True 
	                Surface Type: dural 
	 Number of Smoothing Iterations (if using dural): 30


Enter a custom projection direction as a string (lh,rh,top,bottom,front,back,or custom): If none provided, will default to hemisphere: 
 


::: Loading Mesh data :::
/home/adam2392/hdd/data/neuroimaging/freesurfer_output/umf005/Meshes/rh_dural_trivert.mat
::: Projecting electrodes to mesh :::
rh
::: Done :::


In [27]:
patient.project_electrodes(elecfile_prefix='g', grid=True)

Projection Params: 
	 Grid Name: g.mat 
	 Use Mean Normal: True 
	                Surface Type: dural 
	 Number of Smoothing Iterations (if using dural): 30
Making preproc directory
Normal vectors: [array([-0.95056379, -0.2112423 ,  0.22760751]), array([-0.88967306, -0.36863848,  0.2694207 ]), array([-0.93150767, -0.1686248 ,  0.32227183]), array([-0.8737811 , -0.32401933,  0.3626542 ])]
Projection direction vector:  [-0.9113814042780818, -0.268131228963987, 0.29548855921061334]
::: Loading Mesh data :::
/home/adam2392/hdd/data/neuroimaging/freesurfer_output/umf003/Meshes/lh_dural_trivert.mat
::: Projecting electrodes to mesh :::
[-0.9113814042780818, -0.268131228963987, 0.29548855921061334]
::: Done :::
Moving /home/adam2392/hdd/data/neuroimaging/freesurfer_output/umf003/elecs/individual_elecs/g_orig.mat to /home/adam2392/hdd/data/neuroimaging/freesurfer_output/umf003/elecs/individual_elecs/preproc


#### Creating the **`*_elecs_all.mat`** file

Now we will create our **`'*_elecs_all.mat'`** montage file. This file is a `.mat` file with two structs: **`anatomy`**, which contains information about the montage labels, device origin, and anatomical labels; and **`elecmatrix`**, which contains the electrode coordinate matrix.

**`anatomy`** is structured as a __num_electrodes x 4__ matrix. The first column is the short name abbreviation of the electrode, the second column is the long name, the third column is the electrode type (i.e. depth, strip, or grid).

**`make_elecs_all()`** is an interactive function that creates the **`'*_elecs_all.mat'`** file.

In [13]:
# (short_name, long_name, elec_type, filename)
# umf001
elecs_meta_input = [
    ('btm', 'basal temporal medial', 'strip', 'btm.mat'),
    ('btp', 'basal temporal pole', 'strip', 'btp.mat'),
    ('c', 'grid64', 'grid', 'c.mat'),
]

# # umf 003
# elecs_meta_input = [
#     ('bo', 'basal occipital', 'strip', 'bo.mat'),
#     ('ph', 'hippocampus', 'strip', 'ph.mat'),
#     ('ah', 'anterior hippocampus', 'strip', 'ah.mat'),
#     ('g', 'grid48', 'grid', 'g.mat'),
# ]

# umf004
elecs_meta_input = [
    ('ltp', 'anterior temporal pole', 'depth', 'ltp.mat'),
    ('la', 'right amygdala', 'depth', 'la.mat'),
    ('lofs', 'orbitofrontal', 'depth', 'lofs.mat'),
    ('lph', 'parahippocampal gyrus', 'depth', 'lph.mat'),
    ('lh', 'hippocampus', 'depth', 'lh.mat'),
    ('lmcsm', 'supplementary motor area', 'depth', 'lmcsm.mat'),
    ('ra', 'right amygdala', 'depth', 'ra.mat'),
]

# # umf005
# elecs_meta_input = [
#     ('lpbt', 'posterior basal temporal', 'strip', 'lpbt.mat'),
#     ('labt', 'anterior basal temporal', 'strip', 'labt.mat'),
#     ('llat', 'lateral temporal', 'strip', 'llat.mat'),
#     ('rpbt', 'posterior basal temporal', 'strip', 'rpbt.mat'),
#     ('rabt', 'anterior basal temporal', 'strip', 'rabt.mat'),
#     ('rlat', 'lateral temporal', 'strip', 'rlat.mat'),
# ]



outfile = 'umf004_elecs_all'

print(outfile)

umf004_elecs_all


In [15]:
#creates the TDT_elecs_all.mat file 
patient.make_elecs_all(elecs_meta_input, outfile)

In [16]:
print(scipy.io.loadmat(os.path.join(patient.elecs_dir, f'{outfile}.mat'))['eleclabels'][0:5,:])
print(scipy.io.loadmat(os.path.join(patient.elecs_dir, f'{outfile}.mat'))['elecmatrix'].shape)

[[array(['ltp1'], dtype='<U4')
  array(['anterior temporal pole1'], dtype='<U23')
  array(['depth'], dtype='<U5')]
 [array(['ltp2'], dtype='<U4')
  array(['anterior temporal pole2'], dtype='<U23')
  array(['depth'], dtype='<U5')]
 [array(['ltp3'], dtype='<U4')
  array(['anterior temporal pole3'], dtype='<U23')
  array(['depth'], dtype='<U5')]
 [array(['ltp4'], dtype='<U4')
  array(['anterior temporal pole4'], dtype='<U23')
  array(['depth'], dtype='<U5')]
 [array(['ltp5'], dtype='<U4')
  array(['anterior temporal pole5'], dtype='<U23')
  array(['depth'], dtype='<U5')]]
(66, 3)


In [18]:
#labels the electrodes and saves them to the 'anatomy' struct of the *_elecs_all.mat file
patient.label_elecs(elecfile_prefix=outfile, atlas_surf='desikan-killiany', atlas_depth='desikan-killiany')

/home/adam2392/hdd/data/neuroimaging/freesurfer_output/
Creating labels from the freesurfer annotation file for use in automated electrode labeling
Loading electrode matrix
Loading label parsorbitalis
Loading label parsopercularis
Loading label inferiorparietal
Loading label lingual
Loading label posteriorcingulate
Loading label rostralanteriorcingulate
Loading label pericalcarine
Loading label frontalpole
Loading label lateralorbitofrontal
Loading label postcentral
Loading label insula
Loading label transversetemporal
Loading label temporalpole
Loading label entorhinal
Loading label parahippocampal
Loading label precentral
Loading label medialorbitofrontal
Loading label lateraloccipital
Loading label caudalanteriorcingulate
Loading label middletemporal
Loading label supramarginal
Loading label isthmuscingulate
Loading label superiorfrontal
Loading label superiorparietal
Loading label precuneus
Loading label bankssts
Loading label fusiform
Loading label parstriangularis
Loading label p

array([['ltp1', 'anterior temporal pole1', 'depth',
        'ctx-lh-temporalpole'],
       ['ltp2', 'anterior temporal pole2', 'depth',
        'ctx-lh-inferiortemporal'],
       ['ltp3', 'anterior temporal pole3', 'depth',
        'ctx-lh-inferiortemporal'],
       ['ltp4', 'anterior temporal pole4', 'depth', 'Unknown'],
       ['ltp5', 'anterior temporal pole5', 'depth', 'Unknown'],
       ['ltp6', 'anterior temporal pole6', 'depth', 'Unknown'],
       ['la1', 'right amygdala1', 'depth', 'Unknown'],
       ['la2', 'right amygdala2', 'depth', 'Unknown'],
       ['la3', 'right amygdala3', 'depth', 'ctx-lh-insula'],
       ['la4', 'right amygdala4', 'depth', 'ctx-lh-superiortemporal'],
       ['la5', 'right amygdala5', 'depth', 'Left-Cerebral-White-Matter'],
       ['la6', 'right amygdala6', 'depth', 'ctx-lh-middletemporal'],
       ['la7', 'right amygdala7', 'depth', 'Unknown'],
       ['la8', 'right amygdala8', 'depth', 'Unknown'],
       ['la9', 'right amygdala9', 'depth', 'Unknown']

In [28]:
patient.warp_all(elecfile_prefix=outfile)

Using cvs_avg35_inMNI152 as the template for warps
Computing surface warp
Creating mesh /home/adam2392/hdd/data/neuroimaging/freesurfer_output/cvs_avg35_inMNI152/Meshes/rh_pial_trivert.mat
Making Meshes Directory
Making lh mesh
Making rh mesh
Finding nearest surface vertex for each electrode
Warping each electrode separately:
Warping ch 0
Warping ch 1
Warping ch 2
Warping ch 3
Warping ch 4
Warping ch 5
Warping ch 6
Warping ch 7
Warping ch 8
Warping ch 9
Warping ch 10
Warping ch 11
Warping ch 12
Warping ch 13
Warping ch 14
Warping ch 15
Warping ch 16
Warping ch 17
Warping ch 18
Warping ch 19
Warping ch 20
Warping ch 21
Warping ch 22
Warping ch 23
Warping ch 24
Warping ch 25
Warping ch 26
Warping ch 27
Warping ch 28
Warping ch 29
Warping ch 30
Warping ch 31
Warping ch 32
Warping ch 33
Warping ch 34
Warping ch 35
Warping ch 36
Warping ch 37
Warping ch 38
Warping ch 39
Warping ch 40
Warping ch 41
Warping ch 42
Warping ch 43
Warping ch 44
Warping ch 45
Warping ch 46
Warping ch 47
Warping ch

AttributeError: 'NoneType' object has no attribute 'light_mode'

### Final File Structure

SUBJECTS_DIR/
    * Subj_name/
        * CT/	
        * Meshes/	
        * acpc/	
        * ascii/	
        * cvs/
        * dicom/	
        * elecs/
            -individual_elecs/
        * label/	
        * mri/	
        * scripts/	
        * surf/	

See plotting demo.